# Pointcloud Tutorial

# 1. Basic I/O with PCL-Python

In [3]:
import pcl #Library for Pointcloud
import numpy as np

In [11]:
#1. Read pcl data
pc = pcl.load('./dataset/tabletop.pcd')
print(pc)
print('Type: ', type(pc))

<PointCloud of 202627 points>
Type:  <class 'pcl._pcl.PointCloud'>


In [13]:
#2. Create pcl data

#1) create numpy array
pc_array = np.array([[1,2,3],[3,4,5]], dtype=np.float32)
print(pc_array)

#2) make pointcloud from array
pc = pcl.PointCloud() #Create an object
pc.from_array(pc_array)
print(pc)

[[1. 2. 3.]
 [3. 4. 5.]]
<PointCloud of 2 points>


In [17]:
#3. Write Pointcloud

pcl.save(pc, 'pc2pcd.pcd')

In [22]:
#4. Transform it to numpy array from Pointcloud
pc_array = pc.to_array()

print(pc_array.shape)

# Numpy to Pointcloud
pc_new = pcl.PointCloud()
pc_new.from_array(pc_array)

(2, 3)


In [27]:
cloud = pcl.load('./dataset/tabletop.pcd')
print('Sizes are: ', cloud.size)

Sizes are:  202627


# 2. Data Processing

In [38]:
# Visualize the pointcloud file with Open3D
import open3d as o3d

pcd = o3d.io.read_point_cloud('./dataset/tabletop.pcd')
o3d.visualization.draw_geometries([pcd])

In [47]:
# 1. Downsampling with Voxelization

def do_voxel_grid_downsampling(pcl_data, leaf_size):
    vox = pcl_data.make_voxel_grid_filter() #Set the Voxel Grid
    vox.set_leaf_size(leaf_size, leaf_size, leaf_size) #Set the Leaf size
    result = vox.filter() #Decrease the number of Points
    return result

In [48]:
voxelized_pcd = do_voxel_grid_downsampling(cloud, 0.01)
print(voxelized_pcd)
print(type(voxelized_pcd))

<PointCloud of 55336 points>
<class 'pcl._pcl.PointCloud'>


In [50]:
# 2. ROI Selection with Filters

# 1) passthrough filter
def do_passthrough(pcl_data, filter_axis, axis_min, axis_max):

    #define the filter
    passthrough = pcl_data.make_passthrough_filter()
    
    #set the axis
    passthrough.set_filter_field_name(filter_axis)
    
    #filter limit
    passthrough.set_filter_limits(axis_min, axis_max)
    
    result = passthrough.filter()
    
    return result

In [61]:
cloud = pcl.load('./dataset/lobby.pcd')
cloud_arr = cloud.to_array()
print(cloud)

<PointCloud of 19329 points>


In [57]:
# Visualize the Original Image

pts_path = "./dataset/lobby.pcd"
fragment = o3d.io.read_point_cloud(pts_path)
viz = o3d.visualization.draw_geometries([fragment])

In [75]:
# Passthrough filter execution at x
filter_axis = 'x'
axis_min = 1.0
axis_max = 20.0
cloud_filtered_x = do_passthrough(cloud, filter_axis, axis_min, axis_max)

In [80]:
# Passthrough filter execution at y
filter_axis = 'y'
axis_min = -7.0
axis_max = 5.5
cloud_filtered_y = do_passthrough(cloud, filter_axis, axis_min, axis_max)

In [81]:
pcl.save(cloud_filtered_y, 'pc_filtered_y.pcd')

In [82]:
# Check filtered result
pts_path = "./pc_filtered_y.pcd"
fragment = o3d.io.read_point_cloud(pts_path)
viz = o3d.visualization.draw_geometries([fragment])

In [83]:
#3. Noise Reduction

In [86]:
# Statistical Filtering code
def do_statistical_outlier_filtering(pcl_data, mean_k, tresh):
    
    # Create the filter
    outlier_filter = pcl_data.make_statistical_outlier_filter()
    
    # Set mean value for the filter
    outlier_filter.set_mean_k(mean_k)
    
    # Std dev value set
    outlier_filter.set_std_dev_mul_thresh(tresh)
    
    result = outlier_filter.filter()
    
    return result

In [ ]:
filtered_cloud = do_statistical_outlier_filtering(cloud, 10, 0.001)

In [84]:
#4. RANSAC

In [87]:
def do_ransac_plane_segmentation(pcl_data, pcl_sac_model_plane, pcl_sac_ransac, max_distance):
    '''
    >> Args
    - pcl_sac_model_plane: use to determine plane models
    - pcl_sac_ransac: RANSAC consensus
    '''
    
    seg = pcl_data.make_segmenter() # Declare the segment
    seg.set_model_type(pcl_sac_model_plane)
    seg.set_method_type(pcl_sac_ransac)
    set.set_distance_threshold(max_distance)
    
    return seg

def extract_inlier_outlier(pcl_data, ransac_segmentation):
    inliers, coefficients = ransac_segmentation.segment()
    
    # Recognize inlier v. outlier with negative Boolean
    inlier_object = pcl_data.extract(inliers, negative=False)
    outlier_object = pcl_data.extract(inliers, negative=True)
    
    return inlier_object, outlier_object

In [ ]:
ransac_segmentation = do_ransac_plane_segmentation(cloud, pcl.SACMODEL_PLANE, pcl.SACMODEL_RANSAC, 0.01)
inlier, outliner = extract_inlier_outlier(cloud, ransac_segmentation)